# Capstone Project - The Battle of the Neighborhoods (Week 1 & 2)
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem (Background and Description) <a name="introduction"></a>

Toronto is one of the most populous city in Canada. It is diverse and it has a lot of tourists each year. It is multicultural. It provides lot of business oppourtunities and business friendly environment. It has attracted many different players into the market. The city is a major center for banking and finance, retailing, world trade, transportation, tourism, real estate, new media, traditional media, advertising, legal services, accountancy, insurance, theater, fashion, and food in Canada. This also means that the market is highly competitive. As it is highly developed city so cost of doing business is also one of the highest. Thus, any new business venture or expansion needs to be analysed carefully. The insights derived from analysis will give good understanding of the business environment which help in strategically targeting the market. This will help in reduction of risk. And the Return on Investment will be reasonable.

A restaurant is a business which prepares and serves food and drink to customers in return for money, either paid before the meal, after the meal, or with an open account. Toronto is famous for its excelllent cuisine. It's food culture includes an array of international cuisines influenced by the city's immigrant history.

Central and Eastern European immigrants, Italian immigrants, Jewish immigrants, Chinese and other Asian restaurants, Middle Eastern foods and etc. would be pretty common in this city. To start a new business, someone needs to choose the correct location to start its first venture.If this is successful they can replicate the same in other locations. First move is very important, thereby choice of location is very important.

In this project we will try to find an optimal location for a restaurant in Toronoto. If someone is going to open a restaurant in one of the neighborhoods in Toronto, what options could be available. Since there are lots of restaurants in Toronto, this study try to detect locations that are not already crowded with restaurants. We are also particularly interested in areas with no Italian restaurants in that neighborhood as this restaurant has a lot of customers in Canada. 

We will use our data science powers to generate a few most promissing neighborhoods based on this criteria. We will recommend several neighborhoods and then suggest them to the stakeholders and they can research in more detail and find the best spot for their business idea. 

## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decission are:
* Finding three most common venues in all the neighborhoods in Toronto
* Finding number of existing restaurants in the neighborhood (any type of restaurant)
* Finding the desired neighborhood according to the fact that it has any restaurant near it or not. 

Following data sources will be needed to extract/generate the required information:
* All the common neighborhoods in Toronto will be obtained using **Foursquare API**
* Three most common venues in these neighborhoods in Toronto will be obtained using **Foursquare API**
* Specifiy that which one of these three venues have restaturant in its list and of what type they are. 
* Find the desired neighborhood according to the above criteria and select an appropriate option for opening the Italian restaurant.  

For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. You will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format like the New York dataset. Here is the links for the neighborhoods and its latitude and longitude information:

https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

http://cocl.us/Geospatial_data


## Methodology <a name="methodology"></a>

In this project we will direct our efforts on detecting Neighborhoods of Toronto that have low restaurant density, particularly those with low number of Italian restaurants.

First we specify all the neighborhood of the Toronto from our data and then obtain its latitude and longitude coordinate. Then, we analyze the neighborhoods using the FourSauqre data for all the venues associated to these neighborhoods and then we extract the most three common venues for each neighborhood. Then we consider just the first and second most common venues and check that which neighborhood has restaurants as its common venues which tells us that these areaa are most likely very favoritable for people to eat out. And finally we choose our desire neighborhood for our Italian restaurant business. Below is the method for doing this analysis:

#### Importing libraries


In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize  # tranform JSON file into a pandas dataframe

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
!pip install folium
import folium # map rendering library

     |████████████████████████████████| 93 kB 2.5 MB/s eta 0:00:011


#### Extracting tables from internet

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df_list = pd.read_html(url)
df1=df_list[0]
df1=df1[df1.Borough != 'Not assigned']
df1=df1.reset_index(drop=True)
# Adding the Latitudes and Longitudes to the data
data = pd.read_csv("http://cocl.us/Geospatial_data")
df = pd.merge(df1,data,on='Postal Code')
print('Size of the table is:',df.shape)
df.head()

Size of the table is: (103, 5)


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


#### Extract the Toronto data

In [3]:
df_toronto = df[df['Borough'].str.contains("Toronto")].reset_index(drop=True)
print('Size of the table is:',df_toronto.shape)
df_toronto.head()

Size of the table is: (39, 5)


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


#### Create a map of Toronto with neighborhoods

In [4]:
address = "Toronto, ON"

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto city are {}, {}.'.format(latitude, longitude))
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)
for lat, lng, borough, neighborhood in zip(
        df_toronto['Latitude'], 
        df_toronto['Longitude'], 
        df_toronto['Borough'], 
        df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

The geograpical coordinate of Toronto city are 43.6534817, -79.3839347.


#### Define Foursquare Credentials and Version

In [5]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value


#### Explore all the neighborhoods in Toronto, Top 100 venues that are near them within a radius of 0.5 km

In [6]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
# type your answer here
Toronto_venues = getNearbyVenues(names=df_toronto['Neighbourhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )
print(Toronto_venues.shape)
Toronto_venues.head()

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


#### Analyze Each Neighborhood

In [7]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]
# let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped.head()


,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,Berczy Park,0.000000,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.018182,0.0,0.0,0.000000,0.0
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
2,"Business reply mail Processing Centre, South C...",0.000000,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0,0.0625,0.0625,0.0625,0.125,0.125,0.0625,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
4,Central Bay Street,0.014706,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.014706,0.0,0.0,0.014706,0.0


#### Sort the most common venues in all neighborhoods

In [8]:
# Get sum of all rows in the Dataframe as a Series
total = Toronto_grouped.sum()[1:]
# Get sum of all rows as a new row in Dataframe
total = Toronto_grouped.sum()
total.name = 'Total'
# Assign sum of all rows of DataFrame as a new Row
Toronto_grouped_new = Toronto_grouped.append(total.transpose())
Toronto_grouped_total=Toronto_grouped_new[Toronto_grouped_new.index=='Total']
Toronto_grouped_total

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
Total,"Berczy ParkBrockton, Parkdale Village, Exhibit...",0.292595,0.013333,0.0625,0.0625,0.0625,0.125,0.125,0.0625,0.327783,...,0.21249,0.013333,0.050303,1.273256,0.03,0.221648,0.039412,0.179747,0.139233,0.01


In [9]:
Toronto_grouped_total_sorted=sorted(Toronto_grouped_total, reverse=True)
print('The most common venues are:')
Toronto_grouped_total_sorted[0:20]


The most common venues are:


['Yoga Studio',
 "Women's Store",
 'Wine Bar',
 'Vietnamese Restaurant',
 'Video Game Store',
 'Vegetarian / Vegan Restaurant',
 'Train Station',
 'Trail',
 'Toy / Game Store',
 'Theme Restaurant',
 'Theater',
 'Thai Restaurant',
 'Tea Room',
 'Tanning Salon',
 'Taiwanese Restaurant',
 'Tailor Shop',
 'Swim School',
 'Sushi Restaurant',
 'Supermarket',
 'Strip Club']

#### Let's print each neighborhood along with the top 3 most common venues

In [10]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]
#
num_top_venues = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Berczy Park,Coffee Shop,Farmers Market,Bakery
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot
2,"Business reply mail Processing Centre, South C...",Skate Park,Pizza Place,Brewery
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Coffee Shop
4,Central Bay Street,Coffee Shop,Café,Sandwich Place


#### See which neighborhood has the most 1st and 2nd venue as Restaurant

In [11]:
neighborhoods_venues_res_1st = neighborhoods_venues_sorted[neighborhoods_venues_sorted['1st Most Common Venue'].str.contains("Restaurant")].reset_index(drop=True)
print('Size of the table is:',neighborhoods_venues_res_1st.shape)
neighborhoods_venues_res_1st.head()

Size of the table is: (3, 4)


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,"High Park, The Junction South",Mexican Restaurant,Café,Bar
1,"Kensington Market, Chinatown, Grange Park",Vegetarian / Vegan Restaurant,Mexican Restaurant,Bar
2,"The Danforth West, Riverdale",Greek Restaurant,Coffee Shop,Italian Restaurant


In [12]:
neighborhoods_venues_res_2nd = neighborhoods_venues_sorted[neighborhoods_venues_sorted['2nd Most Common Venue'].str.contains("Restaurant")].reset_index(drop=True)
print('Size of the table is:',neighborhoods_venues_res_2nd.shape)
neighborhoods_venues_res_2nd.head(8)

Size of the table is: (7, 4)


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Church and Wellesley,Coffee Shop,Sushi Restaurant,Japanese Restaurant
1,"Commerce Court, Victoria Hotel",Coffee Shop,Restaurant,Café
2,"Forest Hill North & West, Forest Hill Road Park",Park,Sushi Restaurant,Jewelry Store
3,"Kensington Market, Chinatown, Grange Park",Vegetarian / Vegan Restaurant,Mexican Restaurant,Bar
4,"St. James Town, Cabbagetown",Coffee Shop,Restaurant,Pizza Place
5,Stn A PO Boxes,Coffee Shop,Italian Restaurant,Restaurant
6,"Summerhill West, Rathnelly, South Hill, Forest...",Coffee Shop,American Restaurant,Restaurant


## Analysis <a name="analysis"></a>

The analysis for for the 1st and 2nd most common venues which have restaurants are showing in the above Tables. It's showing that three neighborhoods have a type of restaurants in their first most common venues and six other neighborhoods have restaturants as their 2nd most common venues and one neighborhood has both 1st and 2nd most common venues as restaurants.  

### Cluster Neighborhoods (Run k-means to cluster the neighborhood)

In [13]:
# set number of clusters
kclusters = 3

toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0],
      dtype=int32)

In [14]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = df_toronto
# merge 
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!



,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Pub,Park
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Yoga Studio,Hobby Shop
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Café
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Cocktail Bar
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Pub,Trail


#### let's visualize the resulting clusters

In [15]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Results and Discussion <a name="results"></a>

Our analysis shows that although there is a great number and types of restaurants in Toronto, some neighborhoods might not have enough diversity in the type of restaurants. Highest concentration of restaurants was detected in 9 neighbor hoods as their fist most common venues such as High Park, The Junction South, Kensington Market, Chinatown, Grange Park, The Danforth West, Riverdale adn some other neighborhoods as their 2nd most common venues such as: Church and Wellesley, Commerce Court, Victoria Hotel, Forest Hill North & West, Forest Hill Road Park, St. James Town, Cabbagetown, Stn A PO Boxes, Summerhill West, Rathnelly, South Hill. There is one neighorhood that both 1st and 2nd most common venues include restaurants and it is Kensington Market, Chinatown, Grange Park and the type of restaurant for that is mexician and vegan. We can say that this neighbohood could be really favorable for opening a new restaurant as the density of the people for using restaurants is pretty high in these neighorhoods. 


## Conclusion <a name="conclusion"></a>

Purpose of this project was to identify Toronto areas with low number of restaurants (particularly Italian restaurants) in order to aid stakeholders in narrowing down the search for optimal location for a new Italian restaurant. By calculating the most common venues in each neighborhoods of Toronto and restaurant density distribution in these neighborhoods from Foursquare data we have first identified general boroughs that justify further analysis, and then generated extensive collection of locations which satisfy some basic requirements regarding existing nearby restaurants. Clustering of those locations was then performed in order to create major zones of interest (containing greatest number of potential locations). 

Final decission on optimal restaurant location will be made by stakeholders based on specific characteristics of neighborhoods and locations in every recommended zone, taking into consideration additional factors like attractiveness of each location, levels of noise / proximity to major roads, real estate availability, prices, social and economic dynamics of every neighborhood etc.